In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('output2.csv',encoding ="utf-8")
df.tail()

,Sentence #,Word,Tag
415525,18353,照,O
415526,18353,個,O
415527,18353,X,O
415528,18353,光,O
415529,18353,。,O


In [3]:
df.isnull().sum()

Sentence #    0
Word          0
Tag           0
dtype: int64

In [4]:
df = df.fillna(method='ffill')

In [5]:
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(18354, 2140, 27)

In [6]:
df.head()

,Sentence #,Word,Tag
0,0,醫,O
1,0,師,O
2,0,：,O
3,0,啊,O
4,0,回,O


In [7]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-ID,21
1,B-clinical_event,5
2,B-contact,52
3,B-education,6
4,B-family,50
5,B-location,278
6,B-med_exam,443
7,B-money,126
8,B-name,279
9,B-organization,1


In [8]:
X = df.drop('Tag', axis=1)
X.head()

,Sentence #,Word
0,0,醫
1,0,師
2,0,：
3,0,啊
4,0,回


In [9]:
X.columns

Index(['Sentence #', 'Word'], dtype='object')

In [10]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
X.shape

(415530, 2141)

In [11]:
y = df.Tag.values

In [12]:
classes = np.unique(y)

In [13]:
classes = classes.tolist()
classes

['B-ID',
 'B-clinical_event',
 'B-contact',
 'B-education',
 'B-family',
 'B-location',
 'B-med_exam',
 'B-money',
 'B-name',
 'B-organization',
 'B-others',
 'B-profession',
 'B-time',
 'I-ID',
 'I-clinical_event',
 'I-contact',
 'I-education',
 'I-family',
 'I-location',
 'I-med_exam',
 'I-money',
 'I-name',
 'I-organization',
 'I-others',
 'I-profession',
 'I-time',
 'O']

In [14]:
X.shape, y.shape

((415530, 2141), (415530,))

In [15]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-ID',
 'B-clinical_event',
 'B-contact',
 'B-education',
 'B-family',
 'B-location',
 'B-med_exam',
 'B-money',
 'B-name',
 'B-organization',
 'B-others',
 'B-profession',
 'B-time',
 'I-ID',
 'I-clinical_event',
 'I-contact',
 'I-education',
 'I-family',
 'I-location',
 'I-med_exam',
 'I-money',
 'I-name',
 'I-organization',
 'I-others',
 'I-profession',
 'I-time']

In [16]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [17]:

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,t) for w,t in zip(s['Word'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [18]:
getter = SentenceGetter(df)

In [19]:
sent = getter.get_next()
print(sent)

None


In [20]:
sentences = getter.sentences

In [21]:
sentences

[[('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('啊', 'O'),
  ('回', 'O'),
  ('去', 'O'),
  ('還', 'O'),
  ('好', 'O'),
  ('嗎', 'O'),
  ('？', 'O'),
  ('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('欸', 'O')],
 [('，', 'O'), ('還', 'O'), ('是', 'O'), ('虛', 'O'), ('虛', 'O'), ('的', 'O')],
 [('，', 'O'),
  ('但', 'O'),
  ('。', 'O'),
  ('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('欸', 'O')],
 [('，', 'O'),
  ('真', 'O'),
  ('的', 'O'),
  ('。', 'O'),
  ('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('好', 'O'),
  ('險', 'O'),
  ('好', 'O'),
  ('險', 'O'),
  ('。', 'O'),
  ('坦', 'O'),
  ('白', 'O'),
  ('講', 'O'),
  ('我', 'O'),
  ('剛', 'O'),
  ('回', 'O'),
  ('去', 'O'),
  ('的', 'O'),
  ('時', 'O'),
  ('候', 'O'),
  ('晚', 'O'),
  ('上', 'O'),
  ('還', 'O'),
  ('是', 'O'),
  ('有', 'O'),
  ('盜', 'O'),
  ('汗', 'O'),
  ('。', 'O'),
  ('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('盜', 'O'),
  ('汗', 'O'),
  ('。', 'O'),
  ('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('阿', 'O'),
  ('只', 'O'),
  ('是', 'O'),
  ('前', 'B-time'),
  ('天', 'I-time'),
  (

In [22]:

def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token,label in sent]

def sent2tokens(sent):
    return [token for token,label in sent]

In [23]:

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)

In [25]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [26]:

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

/home/rachel/anaconda3/envs/DL_final/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/rachel/anaconda3/envs/DL_final/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7283958413948431

In [27]:
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

/home/rachel/anaconda3/envs/DL_final/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/rachel/anaconda3/envs/DL_final/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                  precision    recall  f1-score   support

            B-ID       1.00      0.33      0.50         9
B-clinical_event       0.00      0.00      0.00         0
       B-contact       0.78      0.70      0.74        10
     B-education       0.00      0.00      0.00         5
        B-family       0.82      0.69      0.75        13
      B-location       0.82      0.62      0.71        45
      B-med_exam       0.76      0.67      0.71        88
         B-money       0.62      0.65      0.64        23
          B-name       0.83      0.65      0.73        66
  B-organization       0.00      0.00      0.00         1
        B-others       0.00      0.00      0.00         0
    B-profession       1.00      0.06      0.12        16
          B-time       0.76      0.70      0.73       515
            I-ID       1.00      0.50      0.67        30
I-clinical_event       0.00      0.00      0.00         0
       I-contact       0.82      0.64      0.72        42
     I-educat